# **Model Distill**

In [20]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

# ----------------------------------------------------------------------
# A. 定义路径和超参数
# ----------------------------------------------------------------------
# 假设您的 Google Drive 已挂载
BASE_DIR = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT'
BASE_DATA_DIR = os.path.join(BASE_DIR, 'data/processed')
BASE_MODEL_DIR = os.path.join(BASE_DIR, 'models')

# 模型路径
TEACHER_MODEL_PATH = os.path.join(BASE_MODEL_DIR, 'nn_teacher_best_cv.keras')
CHECKPOINT_PATH_DISTILL = os.path.join(BASE_MODEL_DIR, 'best_distilled_model_16_8_D01_LR01.weights.h5')

# KD 核心超参数 (初始探索值)
ALPHA = 0.1      # 软损失权重 (0.3) / 硬损失权重 (0.7)
TEMPERATURE = 10 # 温度参数

# 学生模型最佳内部参数 (来自硬标签训练 F1: 0.7271)
STUDENT_LR = 0.01
STUDENT_DROPOUT = 0.1
FINAL_NN_BASELINE_F1 = 0.7271 # 硬标签 F1 基线
HARD_BASELINE_AUC = 0.6786 # 硬标签 AUC 基线

# ----------------------------------------------------------------------
# B. 数据和模型加载
# ----------------------------------------------------------------------
print("--- 正在加载数据和教师模型 ---")
try:
    # 1. 重新加载所有数据文件
    X_train_full = pd.read_csv(os.path.join(BASE_DATA_DIR, 'X_train_smote.csv'))
    y_train = pd.read_csv(os.path.join(BASE_DATA_DIR, 'y_train_smote.csv')).squeeze()
    X_val = pd.read_csv(os.path.join(BASE_DATA_DIR, 'X_val.csv'))
    y_val = pd.read_csv(os.path.join(BASE_DATA_DIR, 'y_val.csv')).squeeze()

    # 2. 定义学生特征 (行为特征，假设是前10个)
    BEHAV_FEATURES = X_train_full.columns[:10].tolist()
    X_train_student = X_train_full[BEHAV_FEATURES]
    X_val_student = X_val[BEHAV_FEATURES]

    # 3. 加载 Keras 教师模型 (用于生成软标签)
    custom_objects = {'F1Score': tf.keras.metrics.F1Score}
    teacher_model = load_model(TEACHER_MODEL_PATH, custom_objects=custom_objects)

    # 确保 y_train, y_val 是正确的 Keras 格式
    y_val_1d = np.array(y_val)
    y_train_2d = np.expand_dims(np.array(y_train), axis=1)
    y_val_2d = np.expand_dims(y_val_1d, axis=1)

    print(f"✅ 数据集加载成功。学生特征数量: {X_train_student.shape[1]}")
    print(f"✅ 教师模型加载成功。路径: {TEACHER_MODEL_PATH}")

except Exception as e:
    print(f"❌ 严重错误：数据加载或模型加载失败。错误: {e}")
    raise

# ----------------------------------------------------------------------
# C. 知识蒸馏核心：软标签生成和损失函数
# ----------------------------------------------------------------------

# 1. 生成软标签 (Soft Labels)
print("\n--- 正在使用教师模型 (Full Features) 生成软标签 ---")
# 使用 Full Features 预测，获得教师模型的 Sigmoid 概率输出 (0-1)
y_soft_train = teacher_model.predict(X_train_full, verbose=0)
y_soft_val = teacher_model.predict(X_val, verbose=0)

print(f"🎉 软标签生成完毕。训练集软标签 shape: {y_soft_train.shape}")

# 2. 组合标签：[Hard_Label, Soft_Label]
y_train_KD = np.concatenate([y_train_2d, y_soft_train], axis=1)
y_val_KD = np.concatenate([y_val_2d, y_soft_val], axis=1) # 🚀 修正验证集标签维度

print(f"🎉 训练集组合标签 shape: {y_train_KD.shape}")
print(f"🎉 验证集组合标签 shape: {y_val_KD.shape}")


# 3. 自定义 KD 损失函数
def distillation_loss(y_true, y_pred, alpha=ALPHA, temp=TEMPERATURE):
    # y_true 包含两列: [y_hard, y_soft]
    y_hard = y_true[:, 0]  # 真实硬标签 (0或1)
    y_soft = y_true[:, 1]  # 教师的软标签 (Sigmoid 输出)
    y_pred_hard = y_pred[:, 0] # 学生的 Sigmoid 输出 (用于硬损失)

    # 1. 硬标签损失 (Hard Loss): 标准二元交叉熵
    hard_loss = tf.keras.losses.binary_crossentropy(y_hard, y_pred_hard)

    # 2. 软标签损失 (Soft Loss): KL 散度 over Softmax(Logits/T)
    #
    epsilon = tf.keras.backend.epsilon()

    # 将 Sigmoid 概率转换为 Logits
    # Logits = log(p / (1-p))
    logits_T = tf.math.log(tf.clip_by_value(y_soft, epsilon, 1.0 - epsilon) / (1.0 - tf.clip_by_value(y_soft, epsilon, 1.0 - epsilon)))
    logits_S = tf.math.log(tf.clip_by_value(y_pred_hard, epsilon, 1.0 - epsilon) / (1.0 - tf.clip_by_value(y_pred_hard, epsilon, 1.0 - epsilon)))

    # 应用温度 T 进行 Softmax 平滑 (Sigmoid 输出，这里需要将 Logits/T 进行 Softmax 变换以计算 KL 散度)
    # 由于是二分类，Softmax 可以简化为 Sigmoid
    # 但为严谨，我们保持 Logits/Softmax 流程 (即 Logits / T -> Softmax -> KL)
    soft_targets = tf.nn.softmax(logits_T / temp)
    soft_predictions = tf.nn.softmax(logits_S / temp)

    # 计算 KL 散度
    soft_loss = tf.keras.losses.KLDivergence()(soft_targets, soft_predictions)

    # 损失缩放因子 (T^2)
    soft_loss = soft_loss * (temp**2)

    # 3. 组合损失: (1-alpha) * Hard + alpha * Soft
    return (1.0 - alpha) * hard_loss + alpha * soft_loss

# ----------------------------------------------------------------------
# D. 定义 KD 训练模型
# ----------------------------------------------------------------------

# 1. 定义学生模型 (16-8-1, 使用硬标签最佳参数)
nn_distilled_model = Sequential([
    tf.keras.layers.Dense(16, activation='tanh', input_shape=(X_train_student.shape[1],)),
    tf.keras.layers.Dropout(STUDENT_DROPOUT), # D=0.1
    tf.keras.layers.Dense(8, activation='tanh'),
    tf.keras.layers.Dropout(STUDENT_DROPOUT), # D=0.1
    tf.keras.layers.Dense(1, activation='sigmoid', name='student_output')
])

# 2. 编译模型
nn_distilled_model.compile(
    optimizer=Adam(learning_rate=STUDENT_LR), # LR=0.01
    loss=distillation_loss,
    # 验证时只能监控 val_loss 和 val_accuracy
    metrics=['accuracy']
)

# ---------------------------
# E. 训练和评估
# ---------------------------
early_stopping = EarlyStopping(monitor='val_loss', patience=20, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint(CHECKPOINT_PATH_DISTILL, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=0)

print(f"\n--- 🚀 开始知识蒸馏 (T={TEMPERATURE}, alpha={ALPHA}) ---")
print(f"学生结构: 16-8 | LR: {STUDENT_LR} | Dropout: {STUDENT_DROPOUT}")

history_distill = nn_distilled_model.fit(
    X_train_student,
    y_train_KD, # 训练集组合标签
    epochs=300,
    batch_size=32,
    validation_data=(X_val_student, y_val_KD), # 🚀 关键修正：验证集组合标签
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# ---------------------------
# 评估蒸馏效果
# ---------------------------
nn_distilled_model.load_weights(CHECKPOINT_PATH_DISTILL)

y_val_prob_distill = nn_distilled_model.predict(X_val_student, verbose=0).flatten()
y_val_pred_distill = (y_val_prob_distill > 0.5).astype("int32")

f1_distill = f1_score(y_val_1d, y_val_pred_distill)
auc_distill = roc_auc_score(y_val_1d, y_val_prob_distill)
accuracy_distill = accuracy_score(y_val_1d, y_val_pred_distill)
precision_distill = precision_score(y_val_1d, y_val_pred_distill)
recall_distill = recall_score(y_val_1d, y_val_pred_distill)

improvement_f1 = f1_distill - FINAL_NN_BASELINE_F1

print("\n==========================================")
print("       ✨ 知识蒸馏最终性能对比 ✨       ")
print("==========================================")
print(f"学生 NN (硬标签基线 F1): {FINAL_NN_BASELINE_F1:.4f} | AUC: {HARD_BASELINE_AUC:.4f}")

# 输出 KD 结果的表格
df_kd_result = pd.DataFrame({
    'Model': [f'NN_Distilled (T={TEMPERATURE}, a={ALPHA})'],
    'F1-Score (Val)': [f1_distill],
    'Recall (Val)': [recall_distill],
    'Precision (Val)': [precision_distill],
    'Accuracy (Val)': [accuracy_distill],
    'AUC (Val)': [auc_distill]
}).set_index('Model')
print(df_kd_result.round(4).to_markdown())

print(f"\n🎉 知识蒸馏 F1 提升: {improvement_f1*100:.3f} %")

# ---------------------------
# 给出调参建议
# ---------------------------
if auc_distill > HARD_BASELINE_AUC and f1_distill > FINAL_NN_BASELINE_F1:
    print("✅ KD 效果显著，**知识质量和 F1 均有提升**！下一步：尝试更大的 T (如 T=10) 来探索更细致的知识。")
elif f1_distill > FINAL_NN_BASELINE_F1:
    print("✅ F1 略有提升。下一步：尝试更大的 Soft Loss 权重 (如 alpha=0.5) 或更大的 T。")
else:
    print("❌ KD 效果不佳。下一步：尝试减小 T (如 T=2) 或更换 alpha (如 alpha=0.5) 进行探索。")

--- 正在加载数据和教师模型 ---
✅ 数据集加载成功。学生特征数量: 10
✅ 教师模型加载成功。路径: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/nn_teacher_best_cv.keras

--- 正在使用教师模型 (Full Features) 生成软标签 ---
🎉 软标签生成完毕。训练集软标签 shape: (22858, 1)
🎉 训练集组合标签 shape: (22858, 2)
🎉 验证集组合标签 shape: (4183, 2)

--- 🚀 开始知识蒸馏 (T=10, alpha=0.1) ---
学生结构: 16-8 | LR: 0.01 | Dropout: 0.1
Epoch 1/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.4099 - loss: 0.4099 - val_accuracy: 0.4279 - val_loss: 0.3312
Epoch 2/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4226 - loss: 0.3604 - val_accuracy: 0.4170 - val_loss: 0.3501
Epoch 3/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4208 - loss: 0.3604 - val_accuracy: 0.4237 - val_loss: 0.3340
Epoch 4/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4219 - loss: 0.3545 - val_accuracy: 0.4291 - val_loss: 0.3234
Epoch 5/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4232 - loss: 0.3552 - val_accuracy: 0.4261 - val_loss: 0.3283
Epoch 6/300
715

In [22]:
import os

BASE_MODEL_DIR = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models'
print(f"--- 正在列出目录 {BASE_MODEL_DIR} 中的文件 ---")

try:
    files = os.listdir(BASE_MODEL_DIR)
    for f in files:
        print(f)
except FileNotFoundError:
    print("❌ 错误：模型路径不存在或未正确挂载 Google Drive。")
except Exception as e:
    print(f"❌ 发生错误: {e}")

--- 正在列出目录 /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models 中的文件 ---
teacher_catboost_full.cbm
final_student_nn_8_4_distilled.h5
nn_student_distilled.keras
behav_scaler.pkl
nn_teacher_best_cv.keras
nn_student_best_hard_baseline.keras
best_distilled_model_16_8_D01_LR01.weights.h5
best_distilled_model_8_4_FINAL.weights.h5


In [23]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# --- 路径和参数 (F1=0.8815 的最佳配置) ---
BASE_MODEL_DIR = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models'
WEIGHTS_FILENAME = 'best_distilled_model_16_8_D01_LR01.weights.h5'
WEIGHTS_PATH = os.path.join(BASE_MODEL_DIR, WEIGHTS_FILENAME)
STUDENT_DROPOUT = 0.1
STUDENT_LR = 0.01
N_INPUT_FEATURES = 10 # 您的行为特征数量

# 1. 重新构建模型结构 (16-8-1)
nn_final_model = Sequential([
    tf.keras.layers.Dense(16, activation='tanh', input_shape=(N_INPUT_FEATURES,)),
    tf.keras.layers.Dropout(STUDENT_DROPOUT),
    tf.keras.layers.Dense(8, activation='tanh'),
    tf.keras.layers.Dropout(STUDENT_DROPOUT),
    tf.keras.layers.Dense(1, activation='sigmoid', name='student_output')
])

# 2. 编译模型
nn_final_model.compile(optimizer=Adam(learning_rate=STUDENT_LR), loss='binary_crossentropy')

print(f"--- 正在加载最佳权重：{WEIGHTS_PATH} ---")

try:
    # 3. 加载最佳权重
    nn_final_model.load_weights(WEIGHTS_PATH)
    print("✅ 权重加载成功。")

    # 4. 打印参数总结
    print("\n==============================================")
    print("          ✨ 最终模型 (16-8-1) 参数总结 ✨         ")
    print("==============================================")

    # 打印 Summary，显示每一层的参数量
    nn_final_model.summary()

    # 打印每一层参数的详细计算
    print("\n--- 参数量明细 ---")

    # 打印参数计算明细 (与之前的理论计算进行核对)
    # 层 1: 10 -> 16
    W1 = (10 * 16) + 16 # 权重 + 偏置
    # 层 2: 16 -> 8
    W2 = (16 * 8) + 8
    # 层 3: 8 -> 1
    W3 = (8 * 1) + 1

    print(f"输入层(10)到隐藏层1(16): {W1} 个参数")
    print(f"隐藏层1(16)到隐藏层2(8): {W2} 个参数")
    print(f"隐藏层2(8)到输出层(1): {W3} 个参数")
    print(f"总可训练参数: {W1 + W2 + W3}")

except Exception as e:
    print(f"❌ 错误：无法加载权重文件。错误信息: {e}")

--- 正在加载最佳权重：/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/best_distilled_model_16_8_D01_LR01.weights.h5 ---
✅ 权重加载成功。

          ✨ 最终模型 (16-8-1) 参数总结 ✨         


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_40 (Dense)                │ (None, 16)             │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ student_output (Dense)          │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)


--- 参数量明细 ---
输入层(10)到隐藏层1(16): 176 个参数
隐藏层1(16)到隐藏层2(8): 136 个参数
隐藏层2(8)到输出层(1): 9 个参数
总可训练参数: 321


In [24]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ----------------------------------------------------------------------
# A. 评估性能
# ----------------------------------------------------------------------

print("--- 正在使用加载的最佳模型 (F1=0.8815) 评估验证集性能 ---")

# 1. 预测概率值
# 使用 nn_final_model (已加载最佳权重的模型)
y_val_prob_recheck = nn_final_model.predict(X_val_student, verbose=0).flatten()

# 2. 转换为预测类别 (使用标准阈值 0.5)
y_val_pred_recheck = (y_val_prob_recheck > 0.5).astype("int32")

# 3. 计算所有关键指标
f1_recheck = f1_score(y_val_1d, y_val_pred_recheck)
auc_recheck = roc_auc_score(y_val_1d, y_val_prob_recheck)
precision_recheck = precision_score(y_val_1d, y_val_pred_recheck)
recall_recheck = recall_score(y_val_1d, y_val_pred_recheck)
accuracy_recheck = accuracy_score(y_val_1d, y_val_pred_recheck)

# ----------------------------------------------------------------------
# B. 打印最终性能报告
# ----------------------------------------------------------------------

print("\n==============================================")
print("  ✨ 最终部署模型 (16-8-1) 验证集性能确认报告 ✨ ")
print(f"  KD 配置: T={10.0}, alpha={0.1}")
print("==============================================")

df_final_result = pd.DataFrame({
    'Metric': ['F1-Score', 'Recall', 'Precision', 'Accuracy', 'AUC'],
    'Value': [f1_recheck, recall_recheck, precision_recheck, accuracy_recheck, auc_recheck]
})

# 打印表格并与您的记录进行对比
print(df_final_result.set_index('Metric').round(4).to_markdown())

# 最终结论
if np.isclose(f1_recheck, 0.8815, atol=0.0001):
    print("\n✅ **性能确认成功！** 重新计算的 F1-Score 与您记录的 0.8815 完全一致。")
    print("   您可以放心地使用这些数字进行最终报告。")
else:
    print(f"\n⚠️ **警告：** 重新计算的 F1-Score ({f1_recheck:.4f}) 与记录值 (0.8815) 存在细微差异。")
    print("   建议在报告中以本次重现结果为准，并注明原因。")

--- 正在使用加载的最佳模型 (F1=0.8815) 评估验证集性能 ---

  ✨ 最终部署模型 (16-8-1) 验证集性能确认报告 ✨ 
  KD 配置: T=10.0, alpha=0.1
| Metric    |   Value |
|:----------|--------:|
| F1-Score  |  0.8811 |
| Recall    |  0.9012 |
| Precision |  0.8618 |
| Accuracy  |  0.8575 |
| AUC       |  0.9211 |

⚠️ **警告：** 重新计算的 F1-Score (0.8811) 与记录值 (0.8815) 存在细微差异。
   建议在报告中以本次重现结果为准，并注明原因。


In [25]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

# ----------------------------------------------------------------------
# A. 定义路径和超参数
# ----------------------------------------------------------------------
# 最终保存目录
BASE_MODEL_DIR = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models'

# 最佳权重文件路径
WEIGHTS_FILENAME = 'best_distilled_model_16_8_D01_LR01.weights.h5'
WEIGHTS_PATH = os.path.join(BASE_MODEL_DIR, WEIGHTS_FILENAME)

# 最终 Keras 模型保存路径 (使用最终确认的 F1 命名)
FINAL_MODEL_FILENAME = 'FINAL_STUDENT_NN_16_8_F08811.keras'
FINAL_MODEL_SAVE_PATH = os.path.join(BASE_MODEL_DIR, FINAL_MODEL_FILENAME)

# 模型的内部参数
STUDENT_DROPOUT = 0.1
STUDENT_LR = 0.01
N_INPUT_FEATURES = 10

# ----------------------------------------------------------------------
# B. 重新构建模型结构
# ----------------------------------------------------------------------

print("--- 正在重建 16-8-1 学生模型结构 ---")

# 1. 定义学生模型 (16-8-1 结构)
nn_final_model = Sequential([
    tf.keras.layers.Dense(16, activation='tanh', input_shape=(N_INPUT_FEATURES,)),
    tf.keras.layers.Dropout(STUDENT_DROPOUT),
    tf.keras.layers.Dense(8, activation='tanh'),
    tf.keras.layers.Dropout(STUDENT_DROPOUT),
    tf.keras.layers.Dense(1, activation='sigmoid', name='student_output')
])

# 2. 编译模型 (使用标准损失函数)
nn_final_model.compile(
    optimizer=Adam(learning_rate=STUDENT_LR),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ----------------------------------------------------------------------
# C. 加载最佳权重并保存完整模型
# ----------------------------------------------------------------------

try:
    # 3. 加载最佳权重
    nn_final_model.load_weights(WEIGHTS_PATH)
    print(f"✅ 最佳权重从 {WEIGHTS_PATH} 加载成功。")

    # 4. 保存为完整的 Keras 模型文件 (.keras 格式包含结构和权重)
    nn_final_model.save(FINAL_MODEL_SAVE_PATH)
    print(f"🎉 最终模型已成功保存至 Google Drive：")
    print(f"   文件路径：{FINAL_MODEL_SAVE_PATH}")

except Exception as e:
    print(f"❌ 模型保存失败。请检查权重文件路径是否正确，或 Google Drive 路径是否可写。")
    print(f"错误信息: {e}")

--- 正在重建 16-8-1 学生模型结构 ---
✅ 最佳权重从 /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/best_distilled_model_16_8_D01_LR01.weights.h5 加载成功。
🎉 最终模型已成功保存至 Google Drive：
   文件路径：/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/FINAL_STUDENT_NN_16_8_F08811.keras


In [ ]:
print(nn_distilled_model.input_shape)   # (None, 10)
print(nn_distilled_model.output_shape)  # (None, 1)


(None, 10)
(None, 1)


In [17]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

# ----------------------------------------------------------------------
# A. 定义路径和超参数 (使用最终锁定的最佳KD参数)
# ----------------------------------------------------------------------
BASE_DIR = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT'
BASE_DATA_DIR = os.path.join(BASE_DIR, 'data/processed')
BASE_MODEL_DIR = os.path.join(BASE_DIR, 'models')

# 模型路径
TEACHER_MODEL_PATH = os.path.join(BASE_MODEL_DIR, 'nn_teacher_best_cv.keras')
# 专门为 8-4 结构定义 Checkpoint
CHECKPOINT_PATH_DISTILL_8_4 = os.path.join(BASE_MODEL_DIR, 'best_distilled_model_8_4_FINAL.weights.h5')

# KD 核心超参数 (锁定为最优组合)
ALPHA = 0.3       # 软损失权重
TEMPERATURE = 10.0 # 温度参数

# 学生模型内部参数 (保持不变)
STUDENT_LR = 0.01
STUDENT_DROPOUT = 0.1
FINAL_NN_BASELINE_F1 = 0.7271 # 硬标签 F1 基线
HARD_BASELINE_AUC = 0.6786 # 硬标签 AUC 基线

# ----------------------------------------------------------------------
# B. 数据和模型加载 (省略重复加载代码，假设数据已在内存中)
# ----------------------------------------------------------------------
# 注意：此部分代码需要依赖您上一个Notebook中已加载的 X_train_full, y_val, teacher_model 等变量
print("--- 假设数据和教师模型已在内存中，正在准备 KD 标签 ---")
# 假设 y_val_1d, y_train_2d, y_val_2d, y_soft_train, y_soft_val, X_train_student, X_val_student
# 变量已定义并包含正确的值

# ----------------------------------------------------------------------
# C. 知识蒸馏核心：组合标签
# ----------------------------------------------------------------------

# 组合标签：[Hard_Label, Soft_Label]
y_train_KD = np.concatenate([y_train_2d, y_soft_train], axis=1)
y_val_KD = np.concatenate([y_val_2d, y_soft_val], axis=1)

print(f"🎉 训练集组合标签 shape: {y_train_KD.shape}")
print(f"🎉 验证集组合标签 shape: {y_val_KD.shape}")

# ----------------------------------------------------------------------
# D. 定义 KD 训练模型 (核心修改：结构改为 8 -> 4)
# ----------------------------------------------------------------------

# 1. 定义学生模型 (8-4-1 结构)
nn_distilled_model_8_4 = Sequential([
    tf.keras.layers.Dense(8, activation='tanh', input_shape=(X_train_student.shape[1],)), # 8个神经元
    tf.keras.layers.Dropout(STUDENT_DROPOUT), # D=0.1
    tf.keras.layers.Dense(4, activation='tanh'), # 4个神经元
    tf.keras.layers.Dropout(STUDENT_DROPOUT), # D=0.1
    tf.keras.layers.Dense(1, activation='sigmoid', name='student_output')
])

# 2. 编译模型 (使用之前定义的 distillation_loss)
nn_distilled_model_8_4.compile(
    optimizer=Adam(learning_rate=STUDENT_LR), # LR=0.01
    loss=distillation_loss,
    metrics=['accuracy']
)

# ---------------------------
# E. 训练和评估
# ---------------------------
early_stopping = EarlyStopping(monitor='val_loss', patience=20, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint(CHECKPOINT_PATH_DISTILL_8_4, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=0)

print(f"\n--- 🚀 开始结构论证 KD (结构: 8-4 | T={TEMPERATURE}, alpha={ALPHA}) ---")

history_distill = nn_distilled_model_8_4.fit(
    X_train_student,
    y_train_KD, # 训练集组合标签
    epochs=300,
    batch_size=32,
    validation_data=(X_val_student, y_val_KD), # 验证集组合标签
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# ---------------------------
# 评估 8-4 蒸馏效果
# ---------------------------
nn_distilled_model_8_4.load_weights(CHECKPOINT_PATH_DISTILL_8_4)

y_val_prob_distill = nn_distilled_model_8_4.predict(X_val_student, verbose=0).flatten()
y_val_pred_distill = (y_val_prob_distill > 0.5).astype("int32")

f1_distill = f1_score(y_val_1d, y_val_pred_distill)
auc_distill = roc_auc_score(y_val_1d, y_val_prob_distill)
precision_distill = precision_score(y_val_1d, y_val_pred_distill)
recall_distill = recall_score(y_val_1d, y_val_pred_distill)
accuracy_distill = accuracy_score(y_val_1d, y_val_pred_distill)
improvement_f1 = f1_distill - FINAL_NN_BASELINE_F1

print("\n==========================================")
print("       ✨ 结构论证结果 (8-4 vs 16-8) ✨       ")
print("==========================================")
print(f"学生 NN (16-8) 最佳 F1: 0.8815 | AUC: 0.9219")

# 输出 KD 结果的表格
df_kd_result = pd.DataFrame({
    'Model': [f'NN_Distilled (8-4-1)'],
    'F1-Score (Val)': [f1_distill],
    'Recall (Val)': [recall_distill],
    'Precision (Val)': [precision_distill],
    'Accuracy (Val)': [accuracy_distill],
    'AUC (Val)': [auc_distill]
}).set_index('Model')
print(df_kd_result.round(4).to_markdown())

# 最终决策指导
if f1_distill >= 0.8800:
    print("\n✅ 论证结果：**8-4-1 结构效率更高！** F1 性能几乎没有损失。")
    print("   推荐选择 8-4-1 作为最终部署模型。")
elif f1_distill >= 0.8750:
    print("\n⚠️ 论证结果：8-4-1 结构性能有轻微下降。可根据对 F1 的要求决定。")
    print("   如果要求极致性能，选择 16-8-1。")
else:
    print("\n❌ 论证结果：8-4-1 结构容量不足。必须选择 16-8-1 结构。")

--- 假设数据和教师模型已在内存中，正在准备 KD 标签 ---
🎉 训练集组合标签 shape: (22858, 2)
🎉 验证集组合标签 shape: (4183, 2)

--- 🚀 开始结构论证 KD (结构: 8-4 | T=10.0, alpha=0.3) ---
Epoch 1/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.3972 - loss: 0.4888 - val_accuracy: 0.4277 - val_loss: 0.3249
Epoch 2/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4205 - loss: 0.3764 - val_accuracy: 0.4278 - val_loss: 0.3248
Epoch 3/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4238 - loss: 0.3612 - val_accuracy: 0.4271 - val_loss: 0.3241
Epoch 4/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4222 - loss: 0.3666 - val_accuracy: 0.4272 - val_loss: 0.3291
Epoch 5/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4244 - loss: 0.3641 - val_accuracy: 0.4172 - val_loss: 0.3538
Epoch 6/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4228 - loss: 0.3654 - val_accuracy: 0.4240 - val_loss: 0.3304
Epoch 7/300
715/715 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4209 - loss: 0.3682